<a href="https://colab.research.google.com/github/ArturoSbr/MARS/blob/main/MARS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Establecer ambiente local

In [ ]:
# Clonar el repositorio en donde físicamente se encuetra `pyearth`
!pip install git+https://github.com/scikit-learn-contrib/py-earth@v0.2dev

In [ ]:
# Frameworks
import numpy as np
import pandas as pd
# Visualización
import matplotlib.pyplot as plt
# Datos dummy
from sklearn.datasets import load_boston
# Modelos
from sklearn.linear_model import LinearRegression
from pyearth import Earth
# Cargar datos
df = pd.read_csv(load_boston()['filename'], skiprows=1)
# Métrica RMSE
from sklearn.metrics import mean_squared_error as mse

# 1. Funciones de bisagra (hinge functions)
Una función de bisagra está compuesta de dos funciones rectificadoras. Para una observación $x$ y constantes $\beta_1$, $\beta_2$ y $\alpha$, una función de este tipo toma la forma:

$\beta_1 \text{max}(0, x - \alpha)$

ó

$\beta_2 \text{max}(0, \alpha - x)$

Cada una de las funciones dentro de la función de bisgra son iguales a cero antes o después de llegar al *knot*. La función agrega estas dos funciones disjuntas para crear una función lineal por partes:

$f(x; \alpha, \beta_1, \beta_2) = \beta_1 \times \text{max}(0, x - \alpha) +
\beta_2 \times \text{max}(0, \alpha - x)$

In [ ]:
# Definir una función de bisagra
def hinge(x, a, b_1, b_2):
    '''
    Función de bisagra.
    Argumentos:
        x: array-like
            Observaciones de la variable independiente
        a: float
            knot
        b_1: float
            Pendiente de la primera función lineal
        b_2: float
            Pendiente de la segunda función lineal
    '''
    y = []
    for x_i in x:
        y.append(b_1 * max(0, a - x_i) + b_2 * max(0, x_i - a))
    return y

In [ ]:
# Declarar 21 observaciones (-10, -9, ... , 9, 10)
x = np.arange(-10, 11, 1)

In [ ]:
# Cambios en el knot
for a in [-5, 0, 7]:
    plt.plot(x, hinge(x, a=a, b_1=1, b_2=1), label='knot = ' + str(a))
plt.xticks(ticks=x)
plt.ylim(0, 20)
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Cambios en las pendientes
for b_1, b_2 in [(-2, 5), (2, 2), (1, -4)]:
    plt.plot(x, hinge(x, a=0, b_1=b_1, b_2=b_2), label='b_1 = ' + str(b_1) + ', b_2 = ' + str(b_2))
plt.xticks(x)
plt.grid()
plt.legend()
plt.show()

In [ ]:
user_a = float(input('Escoge un knot: '))
user_b1 = float(input('Escoge la pendiente de la primera partición: '))
user_b2 = float(input('Escoge la pendiente de la segunda partición: '))
print('\n\nknot = ', round(user_a, 2), ', b_1 = ',
      round(user_b1, 2), ', b_2 = ', round(user_b2, 2), sep='')

x = np.arange(-100, 100, 1)
y = hinge(x=x, a=user_a, b_1=user_b1, b_2=user_b2)
plt.plot(x, y)
plt.xlabel('x')
plt.ylabel('h(x)')
plt.show()

# 2. Multivariate Adaptive Rregression Spline (MARS)
En esencia, queremos llegar a un modelo con mejor ajuste que una regresión lineal cuando al menos una de las variables independientes tiene una relación no-lineal sobre la variable dependiente.

In [ ]:
x = np.arange(0, 36)
plt.scatter(df['LSTAT'], df['MEDV'], alpha=0.3)
plt.plot(x, 35 - x,
         ls='--', lw=3, color='C1', label='Regresión lineal')
plt.plot(x, 26 + np.array(hinge(x=x, a=6.5, b_1=6, b_2=-0.8)),
         lw=3, color='C3', ls='--', label='MARS')
# Styling
plt.ylim(0, 50)
plt.legend()
plt.xlabel('Viviendas pobres en la vecindad (%)')
plt.ylabel('Precio mediano (miles de USD)')
plt.show()

### 2.1. Reproducción manual del proceso de construcción de variables
Mars propone una función de bisagra usando cada variable independiente y cada punto de corte disponible. Luego corre un modelo lineal y escoge la opción que más reduzca el RMSE.

In [ ]:
# Variable independiente y variable dependiente
x, y = df['LSTAT'].values, df['MEDV'].values

# Propuestas de puntos de corte
for a in [0, 10, 20, 30]:
    # Transformación
    x_hinge = hinge(x=x, a=a, b_1=1, b_2=1)
    # Scatter plots
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(7, 3))
    # x' vs x
    axes[0].scatter(x, x_hinge)
    axes[0].set_xlabel('Variable original')
    axes[0].set_ylabel('Variable transformada')
    # x' vs y
    axes[1].scatter(x_hinge, y)
    axes[1].set_xlabel('Variable transformada')
    axes[1].set_ylabel('Variable de respuesta')
    #Styling
    fig.tight_layout()
    plt.show()

### 2. Modelo MARS
Ajustar un modelo MARS para predecir el valor mediano del precio de las casas en una vecindad (`MEDV`) como función del crimen, edificios industriales, niveles de contaminación, edad promedio de los vecinos, el porcentaje de viviendas pobres en el área, etc.


In [ ]:
df.head()

In [ ]:
# Variables independientes
X = df.loc[:, 'CRIM':'LSTAT']
# Variable dependiente
y = df['MEDV']

# Visualizar relaciones entre cada X_i y el precio
for feature in X.columns:
    plt.scatter(X[feature], y)
    plt.title(feature + ' vs target')
    plt.xlabel(feature)
    plt.ylabel('Precio (USD, miles)')
    plt.show()

In [ ]:
# Declarar modelo MARS
reg = Earth(max_terms=None,
            max_degree=1,
            thresh=0.001,
            allow_linear=True,
            verbose=100)
reg.fit(X, y)

# Declarar modelo lineal
lin = LinearRegression()
lin.fit(X, y)

In [ ]:
print('RMSE de MARS:', np.sqrt(mse(y, reg.predict(X))))
print('RMSE de OLS: ', np.sqrt(mse(y, lin.predict(X))))